In [ ]:
# %% Doc setup
import pandas as pd
import numpy as np
import glob, re, string
from readin import raw_text
import spacy, time, pickle, ginza, ja_ginza, ja_ginza_dict
from itertools import chain
import spacy, time, pickle, ginza, ja_ginza, ja_ginza_dict
import pandas as pd
import numpy as np
import sidetable as stb
from itertools import chain
nlp = spacy.load("ja_ginza")
docs = pickle.load(open("docs.p", "rb"))


In [ ]:
# %% Read in single speaker
jsss = pd.read_csv("data/japanese-single-speaker-speech-dataset/transcript.txt",
                     header=None,
                     sep="|",
                     names=["file","JA","RJ","duration"]).JA


"""
# %% Read in subtitle corpus
subtitles = pd.read_csv("data/japaneseenglish-subtitle-corpus/raw.txt",
                        header=None,
                        sep="\t",
                        names=["EN","JA"]).JA
"""


In [ ]:
# %% Read in knb
knb_gourmet = pd.read_csv("data/knb-corpus/knbc/knbc/corpus2/Gourmet.tsv",
                        header=None,
                        sep="\t",
                        names=["ID","JA1","JA2","JA3","JA4","JA5"])
knb_keitai = pd.read_csv("data/knb-corpus/knbc/knbc/corpus2/Keitai.tsv",
                        header=None,
                        sep="\t",
                        names=["ID","JA1","JA2","JA3","JA4","JA5"])
knb_kyoto = pd.read_csv("data/knb-corpus/knbc/knbc/corpus2/Kyoto.tsv",
                        header=None,
                        sep="\t",
                        names=["ID","JA1","JA2","JA3","JA4","JA5"])
knb_sports = pd.read_csv("data/knb-corpus/knbc/knbc/corpus2/Sports.tsv",
                        header=None,
                        sep="\t",
                        names=["ID","JA1","JA2","JA3","JA4","JA5"])

knb_full = knb_gourmet.append(knb_keitai).append(knb_kyoto).append(knb_sports).fillna(" ")
knb_full["JA"] = knb_full.JA1 + knb_full.JA2 + knb_full.JA3 + knb_full.JA4 + knb_full.JA5
knb = knb_full.JA


In [ ]:
# %% Read in tatoeba
tatoeba = pd.read_csv("data/tatoeba.txt",
                      header=None,
                      sep=";;",
                      names=["EN", "JP", "Source"]).JP

# %% All together now
raw_text = jsss.append(knb).append(tatoeba).astype("unicode").str.strip()#.append(subtitles)
raw_str = raw_text.str.cat(sep=" ").strip()
raw_str = re.sub(r"[^一-龯ぁ-ゞァ-ヶ０-９。、？！]","", raw_str)
raw_str = re.sub(r"[。]","。;", raw_str)
#print(raw_str[:1000])
raw_text = raw_str.split(sep=";")
#print(raw_text[:100])
print(raw_str,  file = open("raw_str.txt", 'w'))

In [ ]:
# %% Model & Docs Spec
start = time.time()

nlp = spacy.load("ja_ginza")
docs= list(nlp.pipe(raw_text, batch_size=100, disable=["ner","textcat"]))

end = time.time()

pickle.dump(docs, open("docs.p", "wb" ))
print(f"NLP Runtime : {end - start} seconds")

In [ ]:
# %% att & chunk dfs

start = time.time()
#docs=docs[:10]
att_df = pd.DataFrame()
count=0
for doc in docs:
    data = pd.DataFrame([[i.text,i.head.text,i.head.pos_,
                        i.head.tag_,i.head.dep_,i.lemma_,i.norm_,
                        i.is_stop,i.pos_,i.tag_,i.dep_,
                        [child.text for child in i.children]] for i in doc])
    att_df=att_df.append(data)
    count +=1
    print(str((len(docs)-count))+" out of "+str(len(docs))+" remaining, "+("{:.0%}".format(count/len(docs)))+" done")
    
att_df.columns=["Token","Head","HeadPOS","HeadTag","HeadDep",
                      "Lemma","Norm","Stop","POS","Tag","Dep","Children"]    
att_df.to_csv("att_df.csv")

end = time.time()
print(f"ATT Runtime : {end - start} seconds")
